The layers of interest are:
Convolutional Layer
Activation Layer
Max-Pool Layer
Fully Connected Layer

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import os
import random
import numpy as np
import random
from tensorflow.python.framework import ops

#-------------------1D-data-------------------------|

In [26]:
# 创建计算图
ops.reset_default_graph
sess = tf.Session()

# 运行参数
data_size = 25  # 数组大小
conv_size = 5   # 卷积核大小
maxpool_size = 5
stride_size = 1 # 位移

# 确保可重现，指定随机数
seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

# 生成 1维数组
data_1d = np.random.normal(size=data_size)
# 占位
x_input_1d = tf.placeholder(shape=[data_size], dtype=tf.float32)

# ----------1d卷积----------
def conv_layer_1d(input_1d,my_filter,stride):
    # tensorflow的"conv2d()"函数只能使用4D数组：[batch#, width, height, channels]
    # 这里使用 batck=1, width=1, height=input length, channel=1，所以必须扩维到4D
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # 使用卷积stride=1, 如果需要增加到2，使用strides=[1,1,2,1]
    convolution_output = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1,1,stride,1], padding="VALID")
    # 降低维度
    conv_output_1d = tf.squeeze(convolution_output)
    return(conv_output_1d)

# 创建卷积滤波器，设定滤波器pattern，filter与input做矩阵乘法
#my_filter = tf.Variable(tf.random_normal(shape=[1,conv_size,1,1]))
my_filter = tf.constant([1.,1.,2.,1.,1.], shape=[1,conv_size,1,1])
# 创建1D卷积层
my_convolution_output = conv_layer_1d(x_input_1d, my_filter,stride=stride_size)

#--------激活函数--------
def activation(input_1d):
    return(tf.nn.relu(input_1d))

# 创建激活层
my_activation_output = activation(my_convolution_output)

#--------池化--------
def max_pool(input_1d, width, stride):
    # 和'conv2d()' 类似, max_pool()需要输入4D arrays.
    # [batch_size=1, width=1, height=num_input, channels=1]
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Perform the max pooling with strides = [1,1,1,1]
    # If we wanted to increase the stride on our data dimension, say by
    # a factor of '2', we put strides = [1, 1, 2, 1]
    # We will also need to specify the width of the max-window ('width')
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, 1, width, 1],
                                 strides=[1, 1, stride, 1],
                                 padding='VALID')
    # 降维
    pool_output_1d = tf.squeeze(pool_output)
    return(pool_output_1d)

my_maxpool_output = max_pool(my_activation_output, width=maxpool_size,stride=stride_size)

#--------Fully Connected--------
def fully_connected(input_layer, num_outputs):
    # First we find the needed shape of the multiplication weight matrix:
    # The dimension will be (length of input) by (num_outputs)
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer),[num_outputs]]))
    # Initialize such weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    # Make the 1D input array into a 2D array for matrix multiplication
    input_layer_2d = tf.expand_dims(input_layer, 0)
    # Perform the matrix multiplication and add the bias
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    # Get rid of extra dimensions
    full_output_1d = tf.squeeze(full_output)
    return(full_output_1d)

my_full_output = fully_connected(my_maxpool_output, 5)

In [38]:
# Run graph
# Initialize Variables
init = tf.global_variables_initializer()
sess.run(init)

feed_dict = {x_input_1d: data_1d}

print('>>>> 1D Data <<<<')

# Convolution Output
print('Input = array of length %d' % (x_input_1d.shape.as_list()[0]))
print(sess.run(x_input_1d, feed_dict=feed_dict))
print('my_filter shape = %s, value as below:' %(my_filter.shape.as_list()))
print(sess.run(my_filter))
print('Convolution w/ filter, length = %d, stride size = %d, results in an array of length %d:' % 
      (conv_size,stride_size,my_convolution_output.shape.as_list()[0]))
print(sess.run(my_convolution_output, feed_dict=feed_dict))

# Activation Output
print('\nInput = above array of length %d' % (my_convolution_output.shape.as_list()[0]))
print('ReLU element wise returns an array of length %d:' % (my_activation_output.shape.as_list()[0]))
print(sess.run(my_activation_output, feed_dict=feed_dict))

# Max Pool Output
print('\nInput = above array of length %d' % (my_activation_output.shape.as_list()[0]))
print('MaxPool, window length = %d, stride size = %d, results in the array of length %d' %
     (maxpool_size,stride_size,my_maxpool_output.shape.as_list()[0]))
print(sess.run(my_maxpool_output, feed_dict=feed_dict))

# Fully Connected Output
print('\nInput = above array of length %d' % (my_maxpool_output.shape.as_list()[0]))
print('Fully connected layer on all 4 rows with %d outputs:' % 
      (my_full_output.shape.as_list()[0]))
print(sess.run(my_full_output, feed_dict=feed_dict))

>>>> 1D Data <<<<
Input = array of length 25
[-0.71239066  0.75376636 -0.04450308  0.45181233  1.3451017   0.5323379
  1.3501879   0.86121136  1.4786857  -1.0453771  -0.788989   -1.261606
  0.5628468  -0.24332625  0.9137407   0.31735092  0.12730329  2.150383
  0.6062887  -0.02677165 -0.9841608   1.1907053   0.9528306  -1.0871816
 -0.14521134]
my_filter shape = [1, 3, 1, 1]
[[[[1.]]

  [[1.]]

  [[2.]]]]
Convolution w/ filter, length = 3, stride size = 1, results in an array of length 23:
[-0.04763046  1.6128879   3.0975127   2.86159     4.5778155   3.6049485
  5.168771    0.24914289 -1.1446694  -4.3575783  -0.9249015  -1.1854117
  2.147002    1.3051163   1.4856982   4.7454205   3.4902637   2.7031283
 -1.3888046   1.3704782   2.1122057  -0.03082728 -0.42477363]

Input = above array of length 23
ReLU element wise returns an array of length 23:
[0.         1.6128879  3.0975127  2.86159    4.5778155  3.6049485
 5.168771   0.24914289 0.         0.         0.         0.
 2.147002   1.3051163

In [35]:
my_filter.shape.as_list()

TensorShape([Dimension(1), Dimension(3), Dimension(1), Dimension(1)])